In [43]:
#! pip install neuroHarmonize
import pandas as pd
import csv
import os
import numpy as np
from neuroHarmonize import harmonizationLearn
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
from matplotlib.font_manager import FontProperties

# Set path to your font
font_path = '/Users/jpillai/Downloads/EB_Garamond/EBGaramond-VariableFont_wght.ttf'
eb_garamond_prop = FontProperties(fname=font_path)

In [45]:
############ UTILS ##################
def process_fMRI_csv_files(csv_directory, atlas):
    # Get a list of CSV filenames in sorted order
    #csv_filenames = sorted([filename for filename in os.listdir(csv_directory) if filename.endswith(".csv")])
    
    #Get a list of csv filenames from csv_directory in lowest to highest order
    csv_filenames = sorted([filename for filename in os.listdir(csv_directory) if filename.endswith(".csv")], key=lambda x: int(x.split('_')[0]))
    print(csv_filenames)
    
    # Create an empty DataFrame to store the results
    result_df = pd.DataFrame()

    # Loop through each CSV filename
    for filename in csv_filenames:
        csv_file_path = os.path.join(csv_directory, filename)
        
        # Read the CSV file using pandas
        df = pd.read_csv(csv_file_path, header=None)
        
        if atlas == 'BNA':
            # Remove the 233rd row and 233rd column from the dataframe
            df = df.drop([233], axis=0)
            df = df.drop([233], axis=1)

        # Flatten the matrix into a single vector
        flattened_data = df.values.flatten()

        # Append to the result dataframe
        result_df = pd.concat([result_df, pd.DataFrame([flattened_data])], ignore_index=True)

        # Print the filename to show progress
        #print(filename)


    return result_df

In [46]:
def harmonizeFunctionalMatrices(data, covariates, atlas, output_dir):

    """
    Harmonizes neuroimaging data using ComBat from the neuroHarmonize package.
    
    This function takes in the path to a directory of CSV files with neuroimaging data,
    performs harmonization using ComBat, reshapes the harmonized data based on the provided
    atlas, and then saves each reshaped matrix back to a separate CSV file in a specified 
    output directory.

    Parameters:
    -----------
    data : str
        Path to a directory containing CSV files with neuroimaging data. Each CSV should contain 
        matrix data flattened into a single vector. The filenames in this directory can be used 
        to derive participant identifiers (PIDN).
        
    covariates : str
        Path to a CSV file containing covariates. The mandatory covariate is 'SITE' and other 
        optional ones include 'AGE_M' & 'SEX'. The 'PIDN' column in this file should match the 
        order of the matrices in the `data` directory.

    atlas : str, one of ['Schaeffer400', 'BNA']
        Specifies the atlas used for the neuroimaging data. Determines the shape of the matrix 
        after reshaping. If the atlas is 'Schaeffer400', the shape is (244, 244). If the atlas is 
        'BNA', the shape is (400, 400).

    output_dir : str
        Path to the desired output directory where harmonized data matrices will be saved as CSV files.

    Returns:
    --------
    None. 
    Harmonized data matrices are saved to individual CSV files in the specified output directory.

    Raises:
    -------
    ValueError:
        If an unsupported atlas is provided.

    Example:
    --------
    > harmonizeFunctionalMatrices('./data/', './covariates.csv', 'BNA', './output/')
    """

    # Load in data
    modelData = pd.read_csv(covariates)
    funcMatrix = process_fMRI_csv_files(data, atlas=atlas)

    # 'funcMatrix' now contains the flattened data from all CSV files in the directory. 
    # Each row corresponds to a flattened vector for each participant.
    funcMatrix = funcMatrix.astype(float)
    funcMatrix = funcMatrix.values

    # Run ComBat (neuroHarmonize) on the data
    func_model, func_adj = harmonizationLearn(funcMatrix, modelData)

    # Convert the harmonized data back into its respective csv
    harmonized_df = pd.DataFrame(func_adj)
    
    # Extract PIDN values directly from the 'covariates' DataFrame
    pidn_list = modelData['PIDN'].tolist()

    # Decide the reshape dimensions based on the atlas
    if atlas == 'Schaeffer400':
        reshape_dims = (400, 400)
    elif atlas == 'BNA':
        reshape_dims = (244, 244)
    else:
        raise ValueError("This atlas has not been integrated into this harmonization yet.")

    # Loop through the DataFrame, unflatten each row into the desired matrix, and save it
    for index, row in harmonized_df.iterrows():
        vector = np.array(row, dtype=float)
        matrix = vector.reshape(reshape_dims)

        # Get the corresponding PIDN from the list
        pidn = pidn_list[index]

        print(pidn)

        # Construct the full path for the output file in the specified directory
        file_name = os.path.join(output_dir, f'{pidn}_HARMONIZED_rois_roi_cor_coef.csv')
        
        matrix_df = pd.DataFrame(matrix)
        
        # Save each matrix with PIDN in the filename to the specified directory
        matrix_df.to_csv(file_name, index=False, header=False)

    print("Harmonization and saving processes are complete!")
    
    return funcMatrix, func_adj

In [48]:
ratl_func = '/Users/jpillai/Documents/ComBat/combat_harmonization_2023/Controls_FINAL_fMRI/data/'
ratl_covariates = '/Users/jpillai/Documents/ComBat/combat_harmonization_2023/Controls_FINAL_fMRI/controls_model_data.csv'
output_func = '/Users/jpillai/Documents/ComBat/combat_harmonization_2023/Controls_FINAL_fMRI/harmonized_data/'

#original_data, harmonized_data = harmonizeCorticalThickness(data=ratl_data, dx='CTRL', covariates=ratl_covariates, output_dir=output_struc, atlas='DK40')
original_func, harmonized_func = harmonizeFunctionalMatrices(data=ratl_func, covariates=ratl_covariates, atlas='BNA', output_dir=output_func)

['1416_2016-08-10_rois_roi_cor_coef.csv', '1619_2018-04-25_rois_roi_cor_coef.csv', '2743_2018-04-26_rois_roi_cor_coef.csv', '3700_2016-08-17_rois_roi_cor_coef.csv', '6677_2017-01-09_rois_roi_cor_coef.csv', '6866_2018-09-21_rois_roi_cor_coef.csv', '6868_2009-06-01_rois_roi_cor_coef.csv', '6976_2008-05-12_rois_roi_cor_coef.csv', '7119_2017-04-03_rois_roi_cor_coef.csv', '7142_2009-04-13_rois_roi_cor_coef.csv', '7397_2016-09-21_rois_roi_cor_coef.csv', '7406_2017-04-03_rois_roi_cor_coef.csv', '7793_2008-10-31_rois_roi_cor_coef.csv', '7802_2016-10-25_rois_roi_cor_coef.csv', '7811_2016-08-10_rois_roi_cor_coef.csv', '8592_2009-04-22_rois_roi_cor_coef.csv', '8612_2009-05-26_rois_roi_cor_coef.csv', '8933_2018-02-23_rois_roi_cor_coef.csv', '9757_2009-11-19_rois_roi_cor_coef.csv', '9761_2016-09-29_rois_roi_cor_coef.csv', '9857_2010-02-19_rois_roi_cor_coef.csv', '9942_2010-06-15_rois_roi_cor_coef.csv', '9954_2010-02-04_rois_roi_cor_coef.csv', '10414_2011-06-23_rois_roi_cor_coef.csv', '10417_2010-03

In [ ]:
# # Select a feature to visualize
# feature_name = "lbankssts"

# # Extract data for the selected feature
# before = original_data[feature_name]
# after = harmonized_data[feature_name]

# # Create a new DataFrame for plotting
# plot_data = pd.DataFrame({
#     "Value": pd.concat([before, after]),
#     "Condition": ["Before"] * len(before) + ["After"] * len(after)
# })

# # Plot the boxplots
# plt.figure(figsize=(10, 6))
# sns.boxplot(x="Condition", y="Value", data=plot_data, width=0.4)
# plt.title(f"Distribution of {feature_name} before and after harmonization", fontproperties=eb_garamond_prop, fontsize=20)
# plt.xlabel("Feature", fontproperties=eb_garamond_prop, fontsize=14)
# plt.ylabel("Value", fontproperties=eb_garamond_prop, fontsize=14)
# plt.xticks(fontproperties=eb_garamond_prop)
# plt.yticks(fontproperties=eb_garamond_prop)
# plt.show()

# #correlation with age, sex --> biological effect remains the same 

In [ ]:
# roi_index = 10  # for example, the 11th region of interest

# # Extract connectivity values for the selected ROI from the original and harmonized matrices
# before_connectivity = original_func[roi_index, :]
# after_connectivity = harmonized_func[roi_index, :]

# plt.figure(figsize=(14, 7))

# # Plot the connectivity values
# plt.plot(before_connectivity, label='Before Harmonization', color='blue', marker='o')
# plt.plot(after_connectivity, label='After Harmonization', color='red', marker='x')

# plt.title(f"Functional Connectivity of ROI {roi_index} before and after Harmonization", fontproperties=eb_garamond_prop, fontsize=20)
# plt.ylabel("Connectivity Strength", fontproperties=eb_garamond_prop, fontsize=14)
# plt.xlabel("Connected Regions", fontproperties=eb_garamond_prop, fontsize=14)
# plt.legend(prop=eb_garamond_prop)
# plt.xticks(fontproperties=eb_garamond_prop)
# plt.yticks(fontproperties=eb_garamond_prop)
# plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# plt.tight_layout()
# plt.show()